# Final project - script

Przygotowałem bibliotekę do przetwarzania danych o dochodach PIT w jednostkach terytorialnych Polski.

In [1]:
%pip install -e calculate_income --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import configparser
import os
import pandas as pd
import numpy as np
from IPython.display import display
import calculate_income as cal_income
import warnings
warnings.filterwarnings('ignore')
%load_ext line_profiler

In [3]:
dir_name = os.path.dirname("script.ipynb")
config = configparser.ConfigParser()
config.read(os.path.join(dir_name, "config.ini"))

['config.ini']

W celu określenia rozdziału klasyfikacji finansowej odpowiedzialnego za PIT użyłem przepisów na rok 2021 pokrywające się w tym zakresie z poprzednimi klasyfikacjami.

Paragraf i rozdział określają rodzaj dochodu ze względu na źródło pochodzenia i rodzaj dochodu:

0010 Wpływy z podatku dochodowego od osób fizycznych\
75621 Udziały gmin w podatkach stanowiących dochód budżetu państw\
75622 Udziały powiatów w podatkach stanowiących dochód budżetu państwa\
75623 Udziały województw w podatkach stanowiących dochód budżetu państwa\
75634 Udziały związków metropolitalnych w podatku dochodowym od osób fizycznych

WK kod województwa\
PK kod powiatu\
GK kod gminy\
GT kod typu gminy (1 miejska, 2 wiejska, 3 miejsko-wiejska)

### Opodatkowany dochód dla jednostek samorządów terytorialnych z roku 2020 oraz 2019.

In [4]:
df_gm = cal_income.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])
df_pow = cal_income.income_comparison(config["data"]["y1_f2"], config["data"]["y2_f2"])
df_mnpp = cal_income.income_comparison(config["data"]["y1_f3"], config["data"]["y2_f3"])
df_pow_mnpp = pd.concat([df_pow, df_mnpp])
df_metrop = cal_income.income_comparison(config["data"]["y1_f4"], config["data"]["y2_f4"])
df_woj = cal_income.income_comparison(config["data"]["y1_f5"], config["data"]["y2_f5"])

In [5]:
%%time
display(df_gm.head())
display(df_pow_mnpp.head())
display(df_metrop)
display(df_woj.head())

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020101,BOLESŁAWIEC,02,01,01,gmina,102838584.0,108377496.0,-5538912.0
020102,BOLESŁAWIEC,02,01,02,gmina,35052648.0,35690532.0,-637884.0
020103,GROMADKA,02,01,03,gmina,9083731.0,9023523.0,60208.0
020104,NOWOGRODZIEC,02,01,04,gmina,24285064.0,24139706.0,145358.0
020105,OSIECZNICA,02,01,05,gmina,14856533.0,15390809.0,-534276.0


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,207575920.0,214452320.0,-6876400.0
020200,DZIERŻONIOWSKI,02,02,00,powiat,208084800.0,213690464.0,-5605664.0
020300,GŁOGOWSKI,02,03,00,powiat,282110336.0,296028192.0,-13917856.0
020400,GÓROWSKI,02,04,00,powiat,55435276.0,56423024.0,-987748.0
020500,JAWORSKI,02,05,00,powiat,101958856.0,102910896.0,-952040.0


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
240000,GÓRNOŚLĄSKO-ZAGŁĘBIOWSKA\nMETROPOLIA,24,00,00,metropolia,7.270829e+09,7.515800e+09,-244970496.0


,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0


CPU times: user 44.2 ms, sys: 5.45 ms, total: 49.7 ms
Wall time: 63.4 ms


Zgodnie z wynikami dochody w jednostkach terytorialnych nieznacznie obniżyły się względem poprzedniego roku.

### Średni dochód opodatkowany dla poszczególnych jednostek samorządu terytorialnego.

Następnie połączyłem te dane z danymi o ludności w poszczególnych Jednostkach Samorządu Terytorialnego i policzyłem średni dochód opodatkowany dla tych jednostek. Wykonanie tych poleceń zajmuje 254 ms, 48.4 ms, 36.8 ms. Różnica wyniku dla gmin związana jest z większą obróbką danych.

In [6]:
%%time
df_gm_mean = cal_income.mean_inc_gm(df_gm, config["data"]["gm_lud"])
display(df_gm_mean.head())

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020101,BOLESŁAWIEC,02,01,01,gmina,102838584.0,108377496.0,-5538912.0,38486.0,26197.093581
020102,BOLESŁAWIEC,02,01,02,gmina,35052648.0,35690532.0,-637884.0,14863.0,23121.401054
020103,GROMADKA,02,01,03,gmina,9083731.0,9023523.0,60208.0,5317.0,16749.328346
020104,NOWOGRODZIEC,02,01,04,gmina,24285064.0,24139706.0,145358.0,15229.0,15633.912785
020105,OSIECZNICA,02,01,05,gmina,14856533.0,15390809.0,-534276.0,7288.0,19985.219120


CPU times: user 210 ms, sys: 8.9 ms, total: 219 ms
Wall time: 254 ms


In [7]:
%%time
df_pow_mnpp_mean = cal_income.mean_inc_pow(df_pow, config["data"]["pow_lud"])
display(df_pow_mnpp_mean.head())

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,207575920.0,214452320.0,-6876400.0,89762.0,22671.708560
020200,DZIERŻONIOWSKI,02,02,00,powiat,208084800.0,213690464.0,-5605664.0,99935.0,20413.737329
020300,GŁOGOWSKI,02,03,00,powiat,282110336.0,296028192.0,-13917856.0,88447.0,31270.561863
020400,GÓROWSKI,02,04,00,powiat,55435276.0,56423024.0,-987748.0,34552.0,15729.424268
020500,JAWORSKI,02,05,00,powiat,101958856.0,102910896.0,-952040.0,49734.0,20098.857826


CPU times: user 36.9 ms, sys: 3.11 ms, total: 40 ms
Wall time: 48 ms


In [8]:
%%time
df_woj_mean = cal_income.mean_inc_woj(df_woj, config["data"]["woj_lud"])
display(df_woj_mean.head())

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch
kod,,,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0,2891321,30472.185429
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0,2061942,23160.883493
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0,2095258,19075.580921
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0,1007145,23664.088402
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0,2437970,26874.528704


CPU times: user 21 ms, sys: 2.72 ms, total: 23.7 ms
Wall time: 34.6 ms


Dla Powiatów i Województw policzyłem wariancję dochodu w podległych jednostkach samorządu terytorialnego i porównałem przewidziany dochód dla tej jednostki ze średnią ważoną jednostek podległych.

In [9]:
df_gm_var = cal_income.income_variance_gm(df_gm_mean, df_pow_mnpp_mean)
display(df_gm_var.head(15))

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch,"(sredni-opod-doch, mean)","(sredni-opod-doch, std)",por_mean
kod,,,,,,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,207575920.0,214452320.0,-6876400.0,89762.0,22671.708560,19849.157338,4107.527598,0.142200
020200,DZIERŻONIOWSKI,02,02,00,powiat,208084800.0,213690464.0,-5605664.0,99935.0,20413.737329,18135.914209,3099.553828,0.125597
020300,GŁOGOWSKI,02,03,00,powiat,282110336.0,296028192.0,-13917856.0,88447.0,31270.561863,26400.874422,9381.843282,0.184452
020400,GÓROWSKI,02,04,00,powiat,55435276.0,56423024.0,-987748.0,34552.0,15729.424268,13858.961335,2905.397621,0.134964
020500,JAWORSKI,02,05,00,powiat,101958856.0,102910896.0,-952040.0,49734.0,20098.857826,18405.868032,3271.831403,0.091981
020600,KARKONOSKI (JELENIOGÓRSKI),02,06,00,powiat,150676304.0,152442784.0,-1766480.0,63333.0,23324.626910,22648.674760,4604.649970,0.029845
020700,KAMIENNOGÓRSKI,02,07,00,powiat,84811496.0,84778840.0,32656.0,42824.0,19416.336405,18061.332856,2721.985596,0.075022
020800,KŁODZKI,02,08,00,powiat,304374944.0,312022848.0,-7647904.0,156283.0,19094.001626,17906.589287,3561.016894,0.066311
020900,LEGNICKI,02,09,00,powiat,137837296.0,135327744.0,2509552.0,55110.0,24520.885276,23393.482048,6199.351330,0.048193


W powiatach, w których jest małe zróżnicowanie dochodu pomiędzy gminami średnia dochodu w gminach jest bliska dochodowi w powiecie. W powiatach, w których jest duże zróżnicowanie dochodu pomiędzy gminami obserwuję, że średni dochód w powiecie różni się od średniej ze średnich dochodów w gminach.

In [10]:
%%time
df_gm_pow_mnpp = pd.concat([df_pow_mnpp_mean, df_gm_mean])
df_pow_var = cal_income.income_variance_pow(df_gm_pow_mnpp, df_woj_mean)
display(df_pow_var)

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku,ludnosc,sredni-opod-doch,"(sredni-opod-doch, mean)","(sredni-opod-doch, std)",por_mean
kod,,,,,,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876928.0,2891321,30472.185429,22432.338308,7698.065915,0.358404
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997440.0,2061942,23160.883493,17381.907021,6712.447943,0.332471
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201243e+09,-124480512.0,2095258,19075.580921,13087.607748,4981.397964,0.457530
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344064.0,1007145,23664.088402,19010.000328,4838.057611,0.244823
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169344.0,2437970,26874.528704,19594.236473,7116.730707,0.371553
120000,MAŁOPOLSKIE,12,00,00,wojewodztwo,9.673090e+09,9.759448e+09,-86358016.0,3410441,27807.024416,18898.483767,7803.779629,0.471389
140000,MAZOWIECKIE,14,00,00,wojewodztwo,2.376420e+10,2.437245e+10,-608249856.0,5425028,42945.836846,21846.303311,13571.255532,0.965817
160000,OPOLSKIE,16,00,00,wojewodztwo,2.261440e+09,2.322817e+09,-61377024.0,976774,22698.168043,18747.340019,3938.924463,0.210741
180000,PODKARPACKIE,18,00,00,wojewodztwo,4.105140e+09,4.196095e+09,-90954752.0,2121229,18973.186197,14894.849191,4424.949993,0.273809


CPU times: user 39.8 ms, sys: 5.31 ms, total: 45.1 ms
Wall time: 107 ms


W przypadku województwa te różnice wyrównują się. Mimo wysokich zróżnicowań pomiędzy dochodami podległych jednostek, średnia z tych dochodów w większości przypadków jest zbliżona do dochodu w danym województwie.

In [11]:
%%time
df = [df_gm, df_pow, df_mnpp, df_metrop, df_woj]
df_all_income = pd.concat(df)

CPU times: user 2.76 ms, sys: 654 µs, total: 3.41 ms
Wall time: 2.97 ms


In [12]:
%%time
with pd.ExcelWriter(config["data"]["results"]) as writer:
    df_all_income.to_excel(writer, sheet_name='doch-jst')
    df_gm_mean.to_excel(writer, sheet_name='gm-srednie-doch')
    df_pow_mnpp_mean.to_excel(writer, sheet_name='pow-mnpp-srednie-doch')
    df_woj_mean.to_excel(writer, sheet_name='woj-srednie-doch')
    df_gm_var.to_excel(writer, sheet_name='pow-stat')
    df_pow_var.to_excel(writer, sheet_name='woj-stat')

CPU times: user 1.47 s, sys: 46.4 ms, total: 1.52 s
Wall time: 1.65 s


Wyniki zapisuję w arkuszach Excel w folderze “results”. Ta operacja zajmuje 2.06 s.

## Profilowanie

Badam funkcję w skrypcie, której wykonanie zajmuje najwięcej czasu.

In [13]:
%lprun -f cal_income.income_comparison cal_income.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])

Timer unit: 1e-06 s

Total time: 3.48511 s
File: /Users/mateusz/Desktop/pit-analysis/calculate_income/calculate_income_jst/jst_income_calc.py
Function: income_comparison at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def income_comparison(file1, file2):
    27         1    1979475.0 1979475.0     56.8      df1 = income_jst(file1)
    28         1    1501207.0 1501207.0     43.1      df2 = income_jst(file2)
    29         1        954.0    954.0      0.0      df1 = df1.rename(columns={'dochod-z-PIT': 'dochod-2019'})
    30         1       1089.0   1089.0      0.0      df2 = df2.rename(columns={'dochod-z-PIT': 'dochod-2020'})
    31         1        167.0    167.0      0.0      df1 = df1["dochod-2019"]
    32         1       1210.0   1210.0      0.0      df = df2.join(df1, how='inner')
    33         1       1004.0   1004.0      0.0      df['roznica-wzgl-poprz-roku'] = (df['dochod-2020'] - df['dochod-2019'])
   

In [14]:
%lprun -f cal_income.income_jst cal_income.income_jst(config["data"]["y1_f1"])

Timer unit: 1e-06 s

Total time: 1.61098 s
File: /Users/mateusz/Desktop/pit-analysis/calculate_income/calculate_income_jst/jst_income_calc.py
Function: income_jst at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def income_jst(input_file):
     6         1    1588853.0 1588853.0     98.6      df = load(input_file)
     7                                               # Participation of territorial self-government units in PIT (percentages).
     8         2          3.0      1.5      0.0      participation_dict = {"75634": 5,
     9         1          1.0      1.0      0.0                            "75623": 1.6,
    10         1          0.0      0.0      0.0                            "75622": 10.25,
    11         1          1.0      1.0      0.0                            "75621": 39.34,
    12                                                                     }
    13                                         

In [15]:
%lprun -f cal_income.load cal_income.load(config["data"]["y1_f1"])

Timer unit: 1e-06 s

Total time: 1.39321 s
File: /Users/mateusz/Desktop/pit-analysis/calculate_income/calculate_income_jst/load_data/load.py
Function: load at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def load(data):
     6         1          2.0      2.0      0.0      try:
     7         2    1384233.0 692116.5     99.4          df = pd.read_excel(data,
     8         1          1.0      1.0      0.0                             skiprows=6,
     9         1          2.0      2.0      0.0                             usecols=[0, 1, 2, 4, 8, 9, 11],
    10         1          1.0      1.0      0.0                             names=["WK", "PK", "GK", "nazwa-JST", "rozdzial", "paragraf", "wplaty"],
    11         2          3.0      1.5      0.0                             dtype={'WK': str,
    12         1          1.0      1.0      0.0                                    'PK': str,
    13         1          1.0   

Wynik profilowania wykazuje, iż najwięcej czasu funkcja poświęca pobraniu danych z pliku.